# Pop2Piano - Training Pipeline 🎹

## Upgraded Version (2026)

This notebook trains the Pop2Piano model with:
- ✅ **Mixed Precision (FP16)** - 2x faster training
- ✅ **Gradient Checkpointing** - 40% less memory
- ✅ **Arabic Maqamat Support** - راست، حجاز، بياتي...
- ✅ **Piano Rules** - Better playability
- ✅ **Resume from Checkpoint** - Continue if interrupted

## Steps:
1. **Setup** - Install dependencies
2. **Clone** - Get latest code
3. **Download** - Get training data
4. **Preprocess** - Prepare audio/MIDI pairs
5. **Train** - Run training loop
6. **Test** - Benchmark results

In [ ]:
# @title 1. Mount Google Drive 💾
from google.colab import drive
import os

drive.mount('/content/drive')

# Create a persistent working directory in your Google Drive
GDRIVE_WORKING_DIR = "/content/drive/MyDrive/Pop2Piano_Workspace"
if not os.path.exists(GDRIVE_WORKING_DIR):
    os.makedirs(GDRIVE_WORKING_DIR)

# Change to the persistent directory
%cd {GDRIVE_WORKING_DIR}

print(f"Switched to persistent directory: {os.getcwd()}")

In [ ]:
# @title 2. Install Dependencies 📦 (Updated 2026)
import os
import sys
import subprocess

print("🔧 Installing updated dependencies...")

# Core ML Libraries (PyTorch 2.x with CUDA)
!pip install -q torch>=2.1.0 torchaudio>=2.1.0 --index-url https://download.pytorch.org/whl/cu121

# Transformers and Lightning (latest)
!pip install -q transformers>=4.40.0 lightning>=2.2.0

# Audio Processing
!pip install -q librosa>=0.10.0 soundfile>=0.12.0 pretty_midi>=0.2.10 note-seq pyFluidSynth

# Configuration & Utilities
!pip install -q omegaconf>=2.3.0 numba>=0.58.0 scipy>=1.11.0

# Training Utilities
!pip install -q tensorboard>=2.15.0 wandb>=0.16.0

# Data Processing
!pip install -q pandas tqdm yt-dlp imageio-ffmpeg

# System packages
!apt-get update -q && apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg

print("✅ All dependencies installed!")

In [ ]:
# @title 3. Setup Essentia (for rhythm extraction) 🪄
import sys

# Try to install essentia, otherwise mock it
try:
    !pip install -q essentia-tensorflow
    import essentia
    print("✅ Essentia installed successfully!")
except:
    print("⚠️ Essentia installation failed. Using mock...")
    from unittest.mock import MagicMock
    from importlib.machinery import ModuleSpec
    
    mock_essentia = MagicMock()
    mock_essentia.__spec__ = ModuleSpec(name='essentia', loader=None)
    
    sys.modules["essentia"] = mock_essentia
    sys.modules["essentia.standard"] = MagicMock()
    
    print("✅ Essentia mocked successfully!")

In [ ]:
# @title 4. Clone Repository 🐙
import os

repo_url = "https://github.com/kareemkamal10/pop2piano.git"
repo_name = "pop2piano"
branch = "upgradeProject"  # Use the upgraded branch with new features

if os.path.exists(repo_name):
    print("Repository already cloned. Pulling latest changes...")
    %cd {repo_name}
    !git fetch origin
    !git checkout {branch}
    !git pull origin {branch}
else:
    print(f"Cloning {repo_url} (branch: {branch})...")
    !git clone -b {branch} {repo_url}
    %cd {repo_name}

print(f"\n✅ Current directory: {os.getcwd()}")
print(f"📌 Branch: {branch}")

# Verify new files exist
new_files = ['piano_rules.py', 'arabic_maqamat.py', 'benchmark.py']
for f in new_files:
    status = "✅" if os.path.exists(f) else "❌"
    print(f"   {status} {f}")

In [ ]:
# @title 4. Run Download Script (CLI / Background Mode) 📥
# This behaves like a GitHub Action: Runs in background, logs to file.
# Prevents browser crash due to excessive output.

print("🚀 Starting Background Download Task...")
print("📄 Logs are being written to: download_log.txt")
print("⚠️ This cell will appear to 'finish' or hang silently - THAT IS GOOD.")
print("👀 Run the NEXT cell to check progress.")

# Redirect stdout and stderr to a file (> download_log.txt 2>&1)
!python download/download.py train_dataset.csv output_dir/ --max_size_gb 15.0 > download_log.txt 2>&1

print("\n✅ Task Completed!")

In [ ]:
# @title 4.1 Monitor Progress (Live Log View) 📺
# Run this cell anytime to see the last 20 lines of the download process.
# Similar to tailing a log in Linux.

!tail -n 20 download_log.txt

# Or to follow it live for a few seconds (uncomment below):
# !timeout 10 tail -f download_log.txt

In [ ]:
# @title 6. Start Training 🏋️ (With New Features)
import os

# Training Configuration
DATA_DIR = "output_dir"  # Directory with preprocessed data
PRECISION = "16-mixed"   # Mixed precision for faster training (options: 32, 16-mixed, bf16-mixed)
EPOCHS = 100             # Number of training epochs
BATCH_SIZE = 8           # Adjust based on GPU memory (Colab T4: 8, A100: 32)
RESUME = None            # Path to checkpoint to resume from (e.g., "checkpoints/last.ckpt")

# Check if data exists
if not os.path.exists(DATA_DIR) or len(os.listdir(DATA_DIR)) == 0:
    print("❌ No data found in output_dir!")
    print("💡 Run the download and preprocess steps first.")
else:
    print(f"📊 Found {len(os.listdir(DATA_DIR))} tracks in {DATA_DIR}")
    print(f"⚙️ Training config: precision={PRECISION}, epochs={EPOCHS}, batch_size={BATCH_SIZE}")
    print("🚀 Starting training...\n")
    
    # Build command
    cmd = f"python train.py --data_dir {DATA_DIR} --precision {PRECISION} --epochs {EPOCHS} --batch_size {BATCH_SIZE}"
    if RESUME:
        cmd += f" --resume {RESUME}"
    
    # Run training
    !{cmd}

In [ ]:
# @title 7. Benchmark & Test 📊
# Run this to test the piano rules and Arabic maqamat

print("🎹 Running benchmark demo...\n")
!python benchmark.py --demo

# Display generated images
from IPython.display import Image, display

print("\n📊 Piano Roll Comparison (Before/After Rules):")
display(Image('benchmark_piano_roll.png'))

print("\n📊 Note Distribution:")
display(Image('benchmark_distribution.png'))

print("\n🎵 Maqam Quantization Demo:")
display(Image('benchmark_maqam.png'))

In [ ]:
# @title 8. Arabic Maqamat Demo 🎵
# Visualize all available Arabic maqamat

print("🎵 Arabic Maqamat Visualization...\n")
!python benchmark.py --maqam-demo

from IPython.display import Image, display
display(Image('maqamat_scales.png'))

# List all maqamat
from arabic_maqamat import list_all_maqamat

print("\n📜 Available Maqamat for Inference:")
for m in list_all_maqamat():
    print(f"   • {m['name_en']:<12} {m['name_ar']:<8} | {m['mood']}")

In [ ]:
# @title 9. Save Model to Google Drive 💾
import shutil
import os
from datetime import datetime

# Checkpoint directory
CHECKPOINT_DIR = "checkpoints"
DRIVE_DIR = "/content/drive/MyDrive/Pop2Piano_Checkpoints"

# Create drive directory if not exists
os.makedirs(DRIVE_DIR, exist_ok=True)

# Find latest checkpoint
if os.path.exists(CHECKPOINT_DIR):
    checkpoints = [f for f in os.listdir(CHECKPOINT_DIR) if f.endswith('.ckpt')]
    if checkpoints:
        # Copy all checkpoints to drive
        for ckpt in checkpoints:
            src = os.path.join(CHECKPOINT_DIR, ckpt)
            dst = os.path.join(DRIVE_DIR, ckpt)
            shutil.copy2(src, dst)
            print(f"✅ Saved: {ckpt}")
        
        print(f"\n📁 Checkpoints saved to: {DRIVE_DIR}")
    else:
        print("❌ No checkpoints found!")
else:
    print("❌ Checkpoint directory not found. Train first!")